In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
#thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
thispath = os.getcwd()
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage

from lib.sych.data_fc_db_raw import DataFCDatabase
import lib.analysis.activity as activity
from lib.common.visualization import merge_image_sequence_movie

%load_ext autoreload
%autoreload 2

In [ ]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
# params['root_path_data'] = './'
params['root_path_data'] = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed'
# params['root_path_data'] = gui_fpath('h5path', './')

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
ds = DataStorage('sych_result_activity.h5')

In [ ]:
mc = MetricCalculator(serial=True, verbose=False)

In [ ]:
print(dataDB.mice)
print(dataDB.dataTypes)
print(dataDB.trialTypeNames)

## Compute mean over time for all mice and condition combinations

In [ ]:
argSweepDict = {
    'datatype' : ['bn_trial', 'bn_session'],
    'intervName' : 'auto',
    'performance' : 'auto'
}

exclQueryLst = []

In [ ]:
activity.compute_mean_interval(dataDB, ds, ['iGO', 'iNOGO'], skipExisting=True, exclQueryLst=exclQueryLst,
                               **argSweepDict)

# 1. Significance

## 1.1 Plot Hit activity by channel

In [ ]:
argSweepDict = {
    'datatype' : ['bn_trial', 'bn_session'],
    'intervName' : 'auto',
    'trialType' : ['iGO', 'iNOGO', 'iMISS', 'iFA'],
    'performance' : 'auto'
}

In [ ]:
activity.activity_brainplot_mouse(dataDB, 'intervName', exclQueryLst=exclQueryLst, vmin=-0.02, vmax=0.02,
                                  **argSweepDict)

In [ ]:
activity.activity_brainplot_mouse(dataDB, 'trialType', exclQueryLst=exclQueryLst, vmin=-0.02, vmax=0.02,
                                  **argSweepDict)

In [ ]:
activity.activity_brainplot_mousephase_subpre(dataDB, exclQueryLst, vmin=-0.02, vmax=0.02,
                                              intervName='auto', trialType='auto')

In [ ]:
activity.activity_brainplot_mousephase_submouse(dataDB, exclQueryLst, vmin=-0.02, vmax=0.02, **argSweepDict)

## 1.2. Accuracy of hit vs cr prediction by mouse, phase

In [ ]:
activity.significance_brainplot_mousephase_byaction(dataDB, ds, performance='naive',
                                                    metric='nlog_pval', minTrials=20, limits=(0, 5))
activity.significance_brainplot_mousephase_byaction(dataDB, ds, performance='expert',
                                                    metric='nlog_pval', minTrials=20, limits=(0, 5))

In [ ]:
activity.classification_accuracy_brainplot_mousephase(dataDB, exclQueryLst, **argSweepDict)

## 1.3. Plot activity movies by chanel

In [ ]:
plotPrefix = activity.activity_brainplot_movie_mousetrialtype(dataDB, exclQueryLst, vmin=-0.02, vmax=0.02,
                                                 haveDelay=False, trialType='auto', performance='auto',
                                                 datatype=['bn_trial', 'bn_session'])

In [ ]:
plotPrefix = 'pics/activity/brainplot_mousetrialtype/movies/'

In [ ]:
for performance in ['naive', 'expert']:
    for datatype in ['bn_trial', 'bn_session']:
        plotSuffix = performance + '_' + datatype + '_'
        merge_image_sequence_movie(plotPrefix + plotSuffix, '.png', 0, 200,
                                   trgPathName=None, deleteSrc=True)

# 2. Consistency

## 2.1. Consistency of channel activity

* DF/F should be independent of channel brightness/opacity, since it normalizes baseline.
* Thus, should be able to compare DFF of different mice directly

## 2.2 Channels significant for Hit/Cr. Overlap by mouse

1. Loop over datatype, interval, session
2. Test each channel for Hit/Cr
3. Count significant sessions for each channel
4. Scatter counts

**TODO**
* Split by Naive/Expert

In [ ]:
activity.plot_consistency_significant_activity_byaction(dataDB, ds, performance='naive',
                                                        dropChannels=[21], metric='accuracy', limits=[0.5, 1])
activity.plot_consistency_significant_activity_byaction(dataDB, ds, performance='expert',
                                                        dropChannels=[21], metric='accuracy', limits=[0.5, 1])

## 2.3. Channels significant for phase. Overlap by mouse

In [ ]:
activity.plot_consistency_significant_activity_byphase(dataDB, ds, performance='naive')
activity.plot_consistency_significant_activity_byphase(dataDB, ds, performance='expert')